In [30]:
# Load packages
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
# Set URL
BBC_URL = "https://www.bbc.com/sport/football/premier-league/table"

In [54]:
# Scrape BBC Sport Premier Leauge Table
def fetch_premier_league_table():
    response = requests.get(BBC_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    if not table:
        print("Could not find the Premier League table on the page.")
        return None
    headers = [th.get_text(strip=True) for th in table.find('thead').find_all('th')]
    rows = []
    for row in table.find('tbody').find_all('tr'):
        cols = [td.get_text(strip=True) for td in row.find_all(['th', 'td'])]
        if len(cols) == len(headers):
            rows.append(dict(zip(headers, cols)))
    df = pd.DataFrame(rows)
    # Identify the team column
    team_col = None
    for col in df.columns:
        if "team" in col.lower():
            team_col = col
            break
    # Extract position and clean team name
    if team_col:
        df['Position'] = df[team_col].str.extract(r'^(\d+)').astype(int)
        df[team_col] = df[team_col].apply(lambda x: re.sub(r'^\d+\.?\s*', '', x))
    return df

def main():
    df = fetch_premier_league_table()
    if df is not None:
        # Move 'Position' to the first column
        cols = list(df.columns)
        if 'Position' in cols:
            cols.insert(0, cols.pop(cols.index('Position')))
            df = df[cols]
        # End on a DataFrame variable
        return df

if __name__ == "__main__":
    prem_table_df = main()

In [55]:
prem_table_df

,Position,Team,Played,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points,"Form, Last 6 games, Oldest first"
0,1,Liverpool,33,24,7,2,75,31,44,79,WResult WinWResult WinWResult WinLResult LossW...
1,2,Arsenal,34,18,13,3,63,29,34,67,WResult WinWResult WinDResult DrawDResult Draw...
2,3,Manchester City,34,18,7,9,66,43,23,61,DResult DrawWResult WinDResult DrawWResult Win...
3,4,Nottingham Forest,33,18,6,9,53,39,14,60,WResult WinWResult WinWResult WinLResult LossL...
4,5,Newcastle United,33,18,5,10,62,44,18,59,WResult WinWResult WinWResult WinWResult WinWR...
5,6,Chelsea,33,16,9,8,58,40,18,57,WResult WinLResult LossWResult WinDResult Draw...
6,7,Aston Villa,34,16,9,9,54,49,5,57,WResult WinWResult WinWResult WinWResult WinWR...
7,8,AFC Bournemouth,33,13,10,10,52,40,12,49,DResult DrawLResult LossLResult LossDResult Dr...
8,9,Fulham,33,13,9,11,48,45,3,48,LResult LossWResult WinLResult LossWResult Win...
9,10,Brighton & Hove Albion,33,12,12,9,53,53,0,48,WResult WinDResult DrawLResult LossLResult Los...
